# EXPLORATORY DATA ANALYSIS

# Python packages

In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.sparse import csr_matrix, issparse
from collections import Counter
import logging

import harmonypy as hm
import scvi
import anndata2ri
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri, numpy2ri, r
from rpy2.robjects.conversion import localconverter

from modules.visualize import *
from modules.deag_tools import *
from modules.utils import *
from MCML.modules import MCML, bMCML
from MCML import tools as tl

# R packages

In [ ]:
pandas2ri.activate()
anndata2ri.activate()
#robjects.r('BiocManager::install("scran", update=FALSE, force=TRUE)')
#ro.r('BiocManager::install("scry", update=FALSE, force=TRUE)')
#ro.r('BiocManager::install("SingleCellExperiment", update=FALSE, force=TRUE)')
ro.r('library(scran)')
ro.r('library(BiocParallel)')
ro.r('library(scry)')
ro.r('library(SingleCellExperiment)')
sc.settings.verbosity = 0
sc.settings.set_figure_params(dpi=80, facecolor="white", frameon=False)
rcb.logger.setLevel(logging.ERROR)

# Import dataset

In [ ]:
# Dictionary mapping sample tags to experimental group name
sample_tag_mapping = {'SampleTag17_flex':'WT-DMSO',
                      'SampleTag18_flex':'3xTg-DMSO',
                      'SampleTag19_flex':'WT-SCDi',
                      'SampleTag20_flex':'3xTg-SCDi',
                      'Undetermined':'Undetermined',
                      'Multiplet':'Multiplet'}
# Load count matrix and convert sample tags to experimental group name
adata = anndata.read_h5ad('data/fede_data/scdi_hypothalamus_count.h5ad')
adata.obs['Sample_Tag'] = adata.obs['Sample_Tag'].map(sample_tag_mapping)
# Load MapMyCells annotations and annotate adata object
anno_df = pd.read_csv("data/fede_data/scdi_hypothalamus_mapping.csv", skiprows=4)
adata = annotate_adata(adata, anno_df)

In [ ]:
# Remove columns that start with 'SampleTag'
columns_to_remove = [col for col in adata.obs.columns if col.startswith('SampleTag')]
adata.obs = adata.obs.drop(columns=columns_to_remove)

# Data analysis

In [ ]:
# Cells distribution by Sample Tag
pie_chart_condition(adata.obs['subclass_name'].values, min_pct=0.02, save_path='figures/pc_subclass.png')

In [ ]:
# Cells distribution by Sample Tag
pie_chart_condition(adata.obs['Sample_Tag'].values, save_path='figures/pc_condition.png')

In [ ]:
# Compute QC metrics
adata.var['mt'] = adata.var_names.str.startswith('mt-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
# Exclusion threshold for mitochondrial read %
adata.obs['high_mt'] = adata.obs['pct_counts_mt'] > 100

In [ ]:
# Display QC metrics
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True, save='violin.png')

In [ ]:
# Display read count distribution of top genes across cells
plot_top_n_distr(adata, top_n=3, save_path='figures/top_n_distr.png')

In [ ]:
# Display QQ plot for negative binomial distribution
plot_top_genes_qq(adata, top_n=3, save_path='figures/top_n_qq.png')

# Quality control

In [ ]:
# Filter out cells with less than N genes with non-zero value
sc.pp.filter_cells(adata, min_genes=200)

In [ ]:
# Filter out genes appearing in less than N cells
sc.pp.filter_genes(adata, min_cells=50)

In [ ]:
# Filter out high_mt cells
adata = adata[~adata.obs['high_mt'], :]
# Filter out multiplets
adata = adata[adata.obs['Sample_Tag'] != "Multiplet", :]

In [ ]:
# Create a layer to store raw counts
adata.layers["counts"] = adata.X.copy()

# Normalization

### Shifted logarithm

In [ ]:
# Perform transformation and store in the log1p_norm layer
scales_counts = sc.pp.normalize_total(adata, target_sum=None, inplace=False)
adata.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)

In [ ]:
display_transformation(adata, 'log1p_norm', save_path='figures/shifted_log_distr.png')

### Scran normalization

In [ ]:
# Perform transformation and store in the scran_normalization layer
adata = scran_normalization(adata)

In [ ]:
display_transformation(adata, 'scran_normalization', save_path='figures/scran_distr.png')

### Pearson residuals

In [ ]:
# Perform transformation and store in the analytic_pearson_residuals layer
adata = pearson_normalization(adata)

In [ ]:
display_transformation(adata, 'analytic_pearson_residuals', save_path='figures/pearson_distr.png')

# Save/Load adata object

In [ ]:
# Write data to specified path
adata.write("data/fede_data/scdi_hypothalamus_normalized.h5ad")

In [ ]:
# Load data from specified path
adata = sc.read(
    filename="data/fede_data/scdi_hypothalamus_normalized.h5ad"
)

# Select features

In [ ]:
adata = select_features(adata)

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=False, layer='scran_normalization')
ax = sns.scatterplot(data=adata.var, x="means", y="dispersions", hue="highly_deviant", s=5)
ax.set_xlim(None, 1.5)
ax.set_ylim(None, 3)
plt.savefig('figures/scran_highly_deviant.png')
plt.show()

In [ ]:
adata.var["highly_variable"] = adata.var["highly_deviant"]

# Dimensionality reduction

## Principal component analysis (PCA)

In [ ]:
# Compute explained variance for first few PCs
show_pc_variance(adata, 'log1p_norm', pc_list=[10,20,50,100])
show_pc_variance(adata, 'scran_normalization', pc_list=[10,20,50,100])
show_pc_variance(adata, 'analytic_pearson_residuals', pc_list=[10,20,50,100])

In [ ]:
# Display scree plot
scree_plot(adata, layer='scran_normalization', save_path='figures/scree_plot.png')

In [ ]:
# PCA gene heatmap - Genes most associated to each PC
plot_top_genes_pca_heatmaps(adata, layer='scran_normalization', n_cells=500, n_top_genes=10, pc_index='10m', n_comps=10, random_seed=42, save_path='figures/pca_gene_heatmap.png')

In [ ]:
# Reduce to N dimensions with PCA
sc.tl.pca(adata, n_comps=10, use_highly_variable=True, layer='scran_normalization')

### Optional - Batch correction

In [ ]:
harmony_out = hm.run_harmony(adata.obsm['X_pca'], adata.obs, 'Sample_Tag')
adata.obsm['X_pca_harmony'] = harmony_out.Z_corr.T

## Multi-class multi-label (MCML)

In [ ]:
adata_mcml = adata[:, adata.var["highly_variable"]].copy()

In [ ]:
subclass_name = adata_mcml.obs['subclass_name'].values.tolist()
sample_tag = adata_mcml.obs['Sample_Tag'].values.tolist()

In [ ]:
mcml = MCML(n_latent = 10, epochs = 100)
latentMCML = mcml.trainTest(adata_mcml.layers['log1p_norm'].toarray(), np.array([subclass_name]), fracNCA = 0.8, silent = True)
mcml.plotLosses(figsize=(10,3),axisFontSize=10,tickFontSize=8, fname='figures/mcml_test.png')

In [ ]:
mcml = MCML(n_latent = 10, epochs = 100)
latentMCML = mcml.fit(adata_mcml.layers['log1p_norm'].toarray(), np.array([subclass_name]), fracNCA = 0.8, silent = True)
mcml.plotLosses(figsize=(10,3),axisFontSize=10,tickFontSize=8, fname='figures/mcml_train.png')

In [ ]:
adata.obsm['X_mcml'] = latentMCML

## scVI

In [ ]:
adata_scvi = adata[:, adata.var["highly_variable"]].copy()
scvi.model.SCVI.setup_anndata(adata_scvi, layer="counts", batch_key="Sample_Tag")
model_scvi = scvi.model.SCVI(adata_scvi)
max_epochs_scvi = int(np.min([round((20000 / adata.n_obs) * 400), 400]))
model_scvi.train(max_epochs=max_epochs_scvi)
adata.obsm["X_scVI"] = model_scvi.get_latent_representation()

# Save/Load adata object

In [ ]:
# Write data to specified path
adata.write("data/fede_data/scdi_hypothalamus_reduced.h5ad")

In [ ]:
# Load data from specified path
adata = sc.read(
    filename="data/fede_data/scdi_hypothalamus_reduced.h5ad"
)

# Clustering

In [ ]:
# Compute cell graph and run leiden clustering algorithm
sc.pp.neighbors(adata, use_rep='X_pca')
sc.tl.leiden(adata, resolution=0.3)

In [ ]:
# Replace cluster id by the cell name most representative of that cluster
assign_unique_cell_type_names(adata, cluster_key='leiden', cluster_types=['class_name', 'subclass_name', 'supertype_name'])

# Visualization

In [ ]:
# Compute 2D UMAP
sc.tl.umap(adata)

### Leiden cluster visualization

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 8))
sc.pl.umap(adata, color='Sample_Tag', ax=axs[0], show=False)
sc.pl.umap(adata, color='cluster_subclass_name', ax=axs[1], show=False)
sc.pl.umap(adata, color='pct_counts_mt', ax=axs[2], show=False)
plt.subplots_adjust(wspace=0.4, hspace=0.4)
plt.tight_layout()
plt.savefig('figures/umap_leiden.png', bbox_inches='tight')
plt.show()

### Cell type visualization by experimental group

In [ ]:
plot_umap(adata, cluster_type='cluster_subclass_name', legend_fontsize=7, save_path='_sample_tag')

### Homogeneity analysis

In [ ]:
class_level, cluster_type = 'subclass_name', 'cluster_subclass_name'

In [ ]:
create_ditto_plot(adata, ['WT-DMSO', '3xTg-DMSO', 'WT-SCDi', '3xTg-SCDi', 'Undetermined'], class_level=class_level, cluster_type=cluster_type, min_cell=50, save_path='figures/all_ditto.png')

# Evaluating embedding quality

In [ ]:
# Calculate the fraction of neighbors for each cell that have the same label as the cell itself.
# Also returns the labels of the neighbors for each embedding method.
subclass_name = adata.obs['subclass_name'].values.tolist()
sample_tag = adata.obs['Sample_Tag'].values.tolist()
# PCA
pca_neighbor_fracs, pca_labels = tl.frac_unique_neighbors(
    adata.obsm['X_pca'], np.array(subclass_name), metric=1, neighbors=30
)

# scVI
scvi_neighbor_fracs, scvi_labels = tl.frac_unique_neighbors(
    adata.obsm['X_scVI'], np.array(subclass_name), metric=1, neighbors=30
)

# MCML
mcml_neighbor_fracs, mcml_labels = tl.frac_unique_neighbors(
    adata.obsm['X_mcml'], np.array(subclass_name), metric=1, neighbors=30
)

# Identify the most common cell types
common_cell_types = [x[0] for x in Counter(adata.obs['subclass_name']).most_common(15)]

# Filter fractions to include only the most common cell types and calculate the mean for each type
pca_neighbor_fracs = {x: np.mean(y) for x, y in pca_neighbor_fracs.items() if x in common_cell_types}
scvi_neighbor_fracs = {x: np.mean(y) for x, y in scvi_neighbor_fracs.items() if x in common_cell_types}
mcml_neighbor_fracs = {x: np.mean(y) for x, y in mcml_neighbor_fracs.items() if x in common_cell_types}

# Combine the results into a single DataFrame
combined_df = pd.DataFrame({
    'PCA': pca_neighbor_fracs,
    'MCML': mcml_neighbor_fracs,
    'scVI': scvi_neighbor_fracs
})


In [ ]:
output_path = "figures/combined_cell_type_data.xlsx"
combined_df.to_excel(output_path)

In [ ]:
combined_df

# Dump data

In [ ]:
adata.write("data/fede_data/scdi_hypothalamus_clustered.h5ad")